In [80]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

In [81]:
from  data_loader import group_dictionary, column_names_dictionary, test_data_no_target,training_data
import pipeline
import cost_function

Drop Split

In [83]:
test_data_no_target = pd.read_csv('data/test_data_no_target.csv', sep=';', decimal=',')
training_data = pd.read_csv('data/training_data.csv', sep=';', decimal=',')

In [85]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()


training_data['Class'] = training_data[['Class']].apply(encoder.fit_transform)

In [87]:
encoder.classes_

array([-1,  0,  1])

In [89]:
X = training_data.drop(columns=['Class','Perform'])  
y = training_data['Class']

Initialize Pipeline and use it

In [90]:
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier


# dt_clf = DecisionTreeClassifier(min_impurity_decrease=0.002, random_state=42)
# dt_clf = RandomForestClassifier(max_depth=14, random_state=42)
# dt_clf = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=10, )



X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


pipe = pipeline.get_pipline(dt_clf,X_train)

pipe.fit(X_train,y_train)

prediction = pipe.predict(X_test)

0:	learn: 1.0812984	total: 99.7ms	remaining: 1m 39s
1:	learn: 1.0657982	total: 134ms	remaining: 1m 6s
2:	learn: 1.0534325	total: 163ms	remaining: 54.2s
3:	learn: 1.0423314	total: 190ms	remaining: 47.4s
4:	learn: 1.0329320	total: 219ms	remaining: 43.5s
5:	learn: 1.0248681	total: 247ms	remaining: 40.9s
6:	learn: 1.0172055	total: 275ms	remaining: 39s
7:	learn: 1.0107584	total: 303ms	remaining: 37.6s
8:	learn: 1.0050514	total: 337ms	remaining: 37.1s
9:	learn: 1.0000901	total: 369ms	remaining: 36.5s
10:	learn: 0.9957669	total: 397ms	remaining: 35.7s
11:	learn: 0.9912664	total: 424ms	remaining: 34.9s
12:	learn: 0.9876821	total: 452ms	remaining: 34.3s
13:	learn: 0.9840514	total: 486ms	remaining: 34.3s
14:	learn: 0.9816766	total: 525ms	remaining: 34.5s
15:	learn: 0.9784344	total: 563ms	remaining: 34.6s
16:	learn: 0.9759647	total: 605ms	remaining: 35s
17:	learn: 0.9738433	total: 643ms	remaining: 35.1s
18:	learn: 0.9718649	total: 678ms	remaining: 35s
19:	learn: 0.9703874	total: 706ms	remaining: 

In [63]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder


# X.drop(columns=['Groups'])
X = pd.concat((X.drop(columns=['Group']), pd.get_dummies(X['Group'])), axis=1)

In [64]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [65]:
model = CatBoostClassifier(
    iterations=1000, depth=6, learning_rate=0.1,
   loss_function='MultiClass', verbose=False)
# model.fit(X_train, y_train)

In [66]:
PARAMS = {
    "n_estimators": [300, 500, 700, 1000],  # [5, 10, 20, 30, 40, 50, 70, 100, 150, 200, 250, 300, 500, 1000],
    "learning_rate": [0.05],
}

In [67]:

model.grid_search(PARAMS, X_train, y_train, cv=5, plot=True, refit=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))


bestTest = 0.9952880688
bestIteration = 86

0:	loss: 0.9952881	best: 0.9952881 (0)	total: 7.37s	remaining: 22.1s

bestTest = 0.9952880688
bestIteration = 86

1:	loss: 0.9952881	best: 0.9952881 (0)	total: 21.8s	remaining: 21.8s

bestTest = 0.9952880688
bestIteration = 86

2:	loss: 0.9952881	best: 0.9952881 (0)	total: 37.5s	remaining: 12.5s

bestTest = 0.9952880688
bestIteration = 86

3:	loss: 0.9952881	best: 0.9952881 (0)	total: 1m	remaining: 0us
Estimating final quality...
Training on fold [0/5]

bestTest = 0.9954472575
bestIteration = 95

Training on fold [1/5]

bestTest = 0.996924727
bestIteration = 64

Training on fold [2/5]

bestTest = 0.9801342312
bestIteration = 221

Training on fold [3/5]

bestTest = 0.9963069809
bestIteration = 111

Training on fold [4/5]

bestTest = 0.9835481916
bestIteration = 175



{'params': {'iterations': 300, 'learning_rate': 0.05},
 'cv_results': defaultdict(list,
             {'iterations': [0,
               1,
               2,
               3,
               4,
               5,
               6,
               7,
               8,
               9,
               10,
               11,
               12,
               13,
               14,
               15,
               16,
               17,
               18,
               19,
               20,
               21,
               22,
               23,
               24,
               25,
               26,
               27,
               28,
               29,
               30,
               31,
               32,
               33,
               34,
               35,
               36,
               37,
               38,
               39,
               40,
               41,
               42,
               43,
               44,
               45,
               46,
               

In [68]:
prediction = model.predict(X_test)

Analyze Prediction

In [93]:
#print(classification_report(y_test, prediction))
print(cost_function.evaluate_error(prediction,y_test))

0.89375


Save Prediction

In [70]:


# Path to save the submission file
submission_file_path = 'submission.txt'

# Open the file in write mode
with open(submission_file_path, 'w') as f:
    # Iterate over each prediction and write it to the file
    for prediction in prediction:
        f.write(f"{prediction}\n")  # Writing each prediction on a new line

print("Submission file created:", submission_file_path)

Submission file created: submission.txt


In [94]:
X_sub = test_data_no_target.copy()

In [95]:
X_sub = pd.concat((X_sub.drop(columns=['Group']), pd.get_dummies(X_sub['Group'])), axis=1)


In [96]:
s = model.predict(X_sub)

In [98]:
fs = encoder.inverse_transform(s.ravel())

In [101]:
# Path to save the submission file
submission_file_path = 'submission_boost1.txt'

# Open the file in write mode
with open(submission_file_path, 'w') as f:
    # Iterate over each prediction and write it to the file
    for prediction in fs.tolist():
        f.write(f"{prediction}\n")  # Writing each prediction on a new line

print("Submission file created:", submission_file_path)

Submission file created: submission_boost1.txt


In [102]:
len(fs)

2000